In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
%matplotlib inline
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:

train = pd.read_hdf('./data/ready_train.hdf')

In [3]:
print(train.shape, '\n', train.info())
train.head(3)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 903653 entries, 0 to 903652
Data columns (total 18 columns):
channelGrouping    903653 non-null int64
visitNumber        903653 non-null int64
bounces            903653 non-null float64
hits               903653 non-null float64
newVisits          903653 non-null float64
pageviews          903653 non-null float64
target             903653 non-null float64
medium             903653 non-null int64
isTrueDirect       903653 non-null int64
isVideoAd          903653 non-null int64
year               903653 non-null float64
dayOfYear          903653 non-null float64
continent          903653 non-null int64
subContinent       903653 non-null int64
country            903653 non-null int64
browser            903653 non-null int64
operatingSystem    903653 non-null int64
deviceCategory     903653 non-null int64
dtypes: float64(7), int64(11)
memory usage: 131.0 MB
(903653, 18) 
 None


,channelGrouping,visitNumber,bounces,hits,newVisits,pageviews,target,medium,isTrueDirect,isVideoAd,year,dayOfYear,continent,subContinent,country,browser,operatingSystem,deviceCategory
0,4,1,1.0,1.0,1.0,1.0,0.0,5,0,0,2016.0,246.0,3,21,204,2,20,0
1,4,1,1.0,1.0,1.0,1.0,0.0,5,0,0,2016.0,246.0,5,1,12,4,7,0
2,4,1,1.0,1.0,1.0,1.0,0.0,5,0,0,2016.0,246.0,4,19,182,2,20,0


In [5]:
#target = train.target.copy()
#train.drop(columns=['target'], inplace=True)
#train = train[['distance']]

train.target = np.log1p(train.target)

#validNonZero = train[train.target != 0].sample(n=2303, random_state=1)
#validZero = train[train.target == 0].sample(n=178427, random_state=1)
#valid = pd.concat((valid_NonZero,validZero))

#train.drop(index=valid.index)
#train.reset_index()

#valid.sample(frac=1., random_state=1, inplace=True).reset_index(drop=True, inplace=True)
train = train.sample(frac=1., random_state=1).reset_index(drop=True)

x_train = train.drop(columns=['target'])
y_train = train.target.copy()

In [10]:


# 	** see this https://github.com/dmlc/xgboost/blob/master/doc/parameter.rst

# 1 - Controling model complexity :
# 	** max_depth, min_child_weight, and gamma
# 	** and those i will tune them for a large scale of values.

# 2 - control model's robusting to noise.
# 	** subsample and colsample_bytree

# 3 - Regularization terms.
# 	** alpha and lambda

# 4 - learning rate and number of rounds.
# 	** eta and num_rounds


params = {

    'max_depth' : 9,
    'min_child_weight' : 3,
    'gamma' : 0,

    'subsample' : 1,
    'colsample_bytree' : 1,

    'reg_alpha' : 0,
    'reg_lambda' : 1,

    'eta' : 0.1,
    'n_estimators':1000,

    'objective' : "reg:linear",
    'eval_metric' : 'rmse',
    'n_jobs' : 4,
    'random_seed' : 10
    }

model = xgb.XGBRegressor(**params)
pg = {}
pg['max_depth'] = [3,5,7,9]
pg['min_child_weight'] = [1,3,5]
#pg['reg_lambda'] = [109,110,111]
#pg['eta'] = [.1, .01, .001]
#pg['n_estimators'] = [1000, 2000]


In [11]:
%%time
#gs = GridSearchCV(model, param_grid=pg, scoring='neg_mean_squared_error', cv=5, verbose=10)
#gs.fit(x_train[:int(1e4)], y_train[:int(1e4)])

model.fit(x_train, y_train)

CPU times: user 47min 21s, sys: 1.17 s, total: 47min 22s
Wall time: 11min 52s


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eta=0.1, eval_metric='rmse', gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=9,
       min_child_weight=3, missing=None, n_estimators=1000, n_jobs=4,
       nthread=None, objective='reg:linear', random_seed=10,
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [9]:
print(gs.best_params_)
print(gs.best_score_)

{'max_depth': 9, 'min_child_weight': 3}
-0.10656448903133582


Linear Model
------------

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as rmse

In [5]:
model = LinearRegression()
model.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [110]:
model.score(x_test, y_test)

0.7649589653732518

In [111]:
rmse(y_test, model.predict(x_test))

6.131354956917073

-------------------
## Submission

In [12]:
test = pd.read_hdf('./data/ready_test.hdf')

In [13]:
pred = model.predict(test)

In [27]:
t = pd.read_csv('./data/fullVisitorId.csv', header=None)
key = t[0].copy()

In [29]:
submission = pd.DataFrame({'fullVisitorId':key, 'PredictedLogRevenue':pred})
submission = submission[['fullVisitorId', 'PredictedLogRevenue']]

submission.to_csv('./data/submission.csv', index=False)

In [16]:
pred = np.expm1(pred)

In [30]:
submission

,fullVisitorId,PredictedLogRevenue
0,6167871330617112363,0.000111
1,0643697640977915618,-0.000122
2,6059383810968229466,0.000298
3,2376720078563423631,-0.000430
4,2314544520795440038,-0.001019
5,4133039884103392367,0.007447
6,4320478850207397557,0.424733
7,5876438247590157131,-0.117095
8,0514591268737702944,-0.000177
9,6430567031531677212,0.000080
